<center><img src="/files/images/DLI_Header.png" /></center>

# 迭代提示开发

在这个 notebook 里，我们通过迭代一组简单的提示来热个身，让我们熟悉一下整个课程都在用的 `transformers` 工作流和 LLaMA-2 模型。

通过对看似简单的提示进行迭代，我们将看到创建**具体**且包含**提示**的提示词的重要性，还将了解如何在面临有挑战性的任务时，为模型提供一点**“思考时间”**。

## 学习目标

完成此 notebook 后，您将能够：
* 使用 `transformers` 工作流从 LLaMA-2 LLM 获得响应。
* 创建**具体**（specific）的提示词。
* 创建为模型提供**“思考时间”**（time to think）的提示词。
* 为模型提供**提示**以指导其响应。

## 视频教程

执行以下单元以加载此 notebook 的视频教程。

In [ ]:
 from IPython.display import HTML

video_url = "https://d36m44n9vdbmda.cloudfront.net/assets/s-fx-12-v1/v2/02-prompting.mp4"

video_html = f"""
<video controls width="640" height="360">
    <source src="{video_url}" type="video/mp4">
    Your browser does not support the video tag.
</video>
"""

display(HTML(video_html))

## 创建 LLaMA-2 工作流

In [ ]:
from transformers import pipeline
model = "TheBloke/Llama-2-13B-chat-GPTQ"

llama_pipe = pipeline("text-generation", model=model, device_map="auto");

## 辅助函数

在此 notebook 中，我们将使用以下函数来支持与 LLM 的交互。

### 生成模型响应

In [ ]:
def generate(prompt, max_length=1024, pipe=llama_pipe, **kwargs):
    """
    Generates a response to the given prompt using a specified language model pipeline.

    This function takes a prompt and passes it to a language model pipeline, such as LLaMA, 
    to generate a text response. The function is designed to allow customization of the 
    generation process through various parameters and keyword arguments.

    Parameters:
    - prompt (str): The input text prompt to generate a response for.
    - max_length (int): The maximum length of the generated response. Default is 1024 tokens.
    - pipe (callable): The language model pipeline function used for generation. Default is llama_pipe.
    - **kwargs: Additional keyword arguments that are passed to the pipeline function.

    Returns:
    - str: The generated text response from the model, trimmed of leading and trailing whitespace.

    Example usage:
    ```
    prompt_text = "Explain the theory of relativity."
    response = generate(prompt_text, max_length=512, pipe=my_custom_pipeline, temperature=0.7)
    print(response)
    ```
    """

    def_kwargs = dict(return_full_text=False, return_dict=False)
    response = pipe(prompt.strip(), max_length=max_length, **kwargs, **def_kwargs)
    return response[0]['generated_text'].strip()

加利福尼亚州首都
--------




我们先从一个非常简单的提示词开始，我们会将其传递给 `generate` 函数，以便从 LLaMA-2 模型中获取响应。在这个提示词的迭代过程中，我们希望模型回复我们加利福尼亚州的首府 Sacramento。

本实验中，我们希望模型仅回复 `"Sacramento"` 这个词。

In [ ]:
prompt = "What is the capital of California?"

print(generate(prompt))

---

模型不明白我们只想要首府城市的名称，而不需要任何其它上下文。这样的话，我们来写一个更**具体**的提示。

In [ ]:
prompt = "What is the capital of California? Only answer this question and do so in as few a words as possible."

print(generate(prompt))

---

有了点改进，但回复的开头仍然有个 `Answer:`。让我们尝试通过向模型提供**提示** `Answer:` 来规避模型的这种行为。这样做有可能就会让模型不再自己输出 `Answer`。

In [ ]:
prompt = "What is the capital of California? Only answer this question and do so in as few a words as possible. Answer: "

print(generate(prompt))

## Sacramento 的元音

这节，我们尝试让模型做点更复杂的事：告诉我们加利福尼亚州首府名字中的所有元音。

正确的答案是 S**a**cr**a**m**e**nt**o** -> **aaeo** -> **aeo**。为了让我自己（和您）更轻松的完成这个任务，我用了以下几个步骤来得出答案。

In [ ]:
prompt = "Tell me the vowels in the capital of California."

print(generate(prompt))

---

当模型需要做多步骤推理时，让它执行多个中间步骤通常会很有帮助，就像要求模型展示其工作过程一样。这种技巧通常被称为给模型“思考时间”。

下面的提示词旨在实现同样的效果，但要求模型先在中间步骤中回复加利福尼亚州首府是什么，然后再给出其中的元音。

In [ ]:
prompt = "Tell me the capital of California, and then tell me all the vowels in it."

print(generate(prompt))

---

现在，我们看到了让模型有**“思考时间”**的效果，下面再次尝试一个稍复杂点的任务：按反向字母顺序给出加利福尼亚州首府的元音。

正确答案是：S**a**cr**a**m**e**nt**o** -> **aaeo** -> **aeo** -> **oea**

In [ ]:
prompt = "Tell me the vowels in the capital of California in reverse alphabetical order?"

print(generate(prompt))

---

为了帮助模型，我们再次提示模型将任务分解为中间步骤，从而为其提供**“思考时间”**。

In [ ]:
prompt = "Tell me the capital of California, and then tell me all the vowels in it, then tell me the vowels in reverse-alphabetical order."

print(generate(prompt))

## 练习

虽然 LLM 不一定是进行数学计算的最佳工具，但作为一项练习，试试用下面的提示词计算 23 和 34 的乘积，然后迭代地开发提示词来获得正确答案。请务必考虑如何让提示词尽可能**具体**，同时为模型提供**“思考时间”**。

如果遇到问题，可以看看下面的参考答案。

In [ ]:
23*34 # Show the actual answer

In [ ]:
prompt = "23x34" # While you and I understand the intention of this prompt, to the model it is not at all **precise**

print(generate(prompt))

### 您的代码




### 参考答案

In [ ]:
prompt = "Calculate the product of 23 and 34. Use the steps typical of long multiplication and show your work."

print(generate(prompt))

## 关键概念回顾

此 notebook 中介绍了这几个关键概念：
* **具体**：尽可能明确的指导 LLM。
* **提示**：用于指导响应，通常是为了防止在响应中包含提示本身。
* **“思考时间”**：一种要求模型执行多步骤并展示其工作来提升 LLM 响应质量的方式（通常计算任务会涉及到）。

重启内核
----




为下一个 notebook 释放 GPU 显存，请运行以下单元重启内核。

In [ ]:
from IPython import get_ipython

get_ipython().kernel.do_shutdown(restart=True)